In [1]:
from libs.spark_session import Spark
from libs.utils import *
from libs.ingestors import Ingestor,IngestorCDC

In [2]:
spark = Spark().spark

In [3]:
catalog = 'bronze'
database = 'postgres'
table = 'financeiro.clientes'
data_format = 'parquet'
source_as_string = 'true'
id_field = 'cliente_id'
timestamp_field = 'modificado_em'

table_aux = table.replace(".", "_")
table_formatted = f"{database}_{table_aux}"

fullload_source = f's3a://raw/{database}/full_load/{table}/'
cdc_source = f's3a://raw/{database}/cdc/{table}/'
checkpoint_location = f's3a://raw/{database}/cdc/{table}_checkpoint/'

In [4]:
if not table_exists(spark,catalog,table_formatted):
    print(f"ingesting table {table_formatted}")
    ingestor = Ingestor(
    spark,
    catalog,
    database,
    table,
    data_format
    )
    ingestor.execute(raw_source=fullload_source)
else:
    print(f"ingesting table using cdc {table_formatted}")
    ingestor = IngestorCDC(
        spark,
        catalog,
        database,
        table,
        data_format,
        source_as_string,
        checkpoint_location,
        id_field,
        timestamp_field
    )
    stream = ingestor.execute(raw_source=cdc_source)

ingesting table using cdc postgres_financeiro_clientes


In [5]:
df = spark.sql(f"select count(*) from {catalog}.{table_formatted}").show()

+--------+
|count(1)|
+--------+
|   22000|
+--------+

